In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot

### 01. 輸入文字資料

我們從 PTT 找到一些評論的資料, 數量很少只是當練習使用。我們把資料存成 `.csv` 檔, 但因為文字有時會有半型逗號, 因此我們用 `*` 號分隔。格式如下 (正評):

    一則評論*1
    
負評自然就是 0。

In [2]:
df = pd.read_csv('data/review.csv', sep='*')

In [3]:
df.head()

,評論,正負評
0,蠻穩定的 行車記錄器就是要穩 其他其次,1
1,流媒體不錯 晚上又清楚,1
2,最近有裝 A129 pro duo，畫質不錯，之前mio 791d用兩年也沒遇到問題,1
3,mio 後鏡頭一直斷線 拉線要很注意 很麻煩,0
4,晚上只有cansonic z3這種有望遠鏡的才拍的到,1


### 02. 找出中文的部份

對不同任務, 我們可能會有不同考量。這裡因為我們只是想知道這則評論是正評還是負評, 對於牌子啦、型號等等我們就不保留。

用 Regular Expression 很容易做到。在 Python 裡要實現 regular expression 可以用 `re` 套件。

In [4]:
import re

使用上我們就是先規定一下一個 pattern, 就是要找的文字需符合什麼要求, 中文文字的範圍在 unicode 中是:

    u4E00 - u9FD5
    
於是我們說看到這, 而後面也符合這個就一起要下來。這樣還能自動斷句。

In [5]:
patn=re.compile(r"[\u4E00-\u9FD5]+")

我們拿一則評論試試。

In [6]:
X = df['評論'][3]

In [7]:
X

'mio 後鏡頭一直斷線 拉線要很注意 很麻煩'

用我們的 pattern 試試能不能只留下中文的部份。

In [8]:
patn.findall(X)

['後鏡頭一直斷線', '拉線要很注意', '很麻煩']

成功了! 我們應用到所有的句子上。

In [9]:
for review in df['評論']:
    print(patn.findall(review))

['蠻穩定的', '行車記錄器就是要穩', '其他其次']
['流媒體不錯', '晚上又清楚']
['最近有裝', '畫質不錯', '之前', '用兩年也沒遇到問題']
['後鏡頭一直斷線', '拉線要很注意', '很麻煩']
['晚上只有', '這種有望遠鏡的才拍的到']
['邁不錯']
['現在新的', '錄一錄會一直出現記憶卡損壞的語音就停止了', '要重裝記憶卡不知道是只有這台']
['邁後視鏡含夜光後視', '六月裝家裡兩台老車', '後視都裝在車外', '目前都正常', '便宜好選擇']
['我要跟你說', '他的導航是廢渣', '要用他的導航才能顯示左轉右轉和路名']
['以前用很久', '放', '還行']
['不過用久了', '像深灰色款的會慢慢變淡一點']
['我用了快五年了', '持續使用中']
['我覺得很好用']
['用過一款', '吋包超過十年', '很滿意']
['個人頗推這個牌子好看又耐用']
['上次買到', '折以後約', '年還在用']
['這個沒有', '充電呀']
['我剛好有買這款', '因為有三個', '插槽才買的', '質感還算不錯', '平常會外接螢幕使用', '會熱但不至於到燙', '跟充電都不會影響']


### 03. 化資料為我們要的形式

這時我們又發現, 有個新的問題, 那就是到底要不要考慮句子的分隔? 還是把整句都連起來? 我們這裡用空白把連起來。

In [10]:
x_tmp = []

for review in df['評論']:
    sen_list = patn.findall(review)
    sen = ' '.join(sen_list)
    x_tmp.append(sen)

我們可以看到, 目前 `x_tmp` 裡就是我們要的句子的樣子了。

In [11]:
x_tmp

['蠻穩定的 行車記錄器就是要穩 其他其次',
 '流媒體不錯 晚上又清楚',
 '最近有裝 畫質不錯 之前 用兩年也沒遇到問題',
 '後鏡頭一直斷線 拉線要很注意 很麻煩',
 '晚上只有 這種有望遠鏡的才拍的到',
 '邁不錯',
 '現在新的 錄一錄會一直出現記憶卡損壞的語音就停止了 要重裝記憶卡不知道是只有這台',
 '邁後視鏡含夜光後視 六月裝家裡兩台老車 後視都裝在車外 目前都正常 便宜好選擇',
 '我要跟你說 他的導航是廢渣 要用他的導航才能顯示左轉右轉和路名',
 '以前用很久 放 還行',
 '不過用久了 像深灰色款的會慢慢變淡一點',
 '我用了快五年了 持續使用中',
 '我覺得很好用',
 '用過一款 吋包超過十年 很滿意',
 '個人頗推這個牌子好看又耐用',
 '上次買到 折以後約 年還在用',
 '這個沒有 充電呀',
 '我剛好有買這款 因為有三個 插槽才買的 質感還算不錯 平常會外接螢幕使用 會熱但不至於到燙 跟充電都不會影響']

### 04. 計算字出現的頻率

最後我們要做的是計算每個字出現的次數, 越常出現的字編號越前面。還有那空白我們準備把它編號 0, 就不參加排序。

我們先用一個例子小試身手。

In [12]:
X = x_tmp[3]

In [13]:
X

'後鏡頭一直斷線 拉線要很注意 很麻煩'

我們不要空白的話是這樣... (等於回到以前)

In [14]:
X.split()

['後鏡頭一直斷線', '拉線要很注意', '很麻煩']

自然全部合起來就是沒有空白的了!

In [15]:
''.join(X.split())

'後鏡頭一直斷線拉線要很注意很麻煩'

現在我們來做全部文字放在一起的字串!

In [16]:
egg = ''.join(''.join(x_tmp).split())

In [17]:
egg

'蠻穩定的行車記錄器就是要穩其他其次流媒體不錯晚上又清楚最近有裝畫質不錯之前用兩年也沒遇到問題後鏡頭一直斷線拉線要很注意很麻煩晚上只有這種有望遠鏡的才拍的到邁不錯現在新的錄一錄會一直出現記憶卡損壞的語音就停止了要重裝記憶卡不知道是只有這台邁後視鏡含夜光後視六月裝家裡兩台老車後視都裝在車外目前都正常便宜好選擇我要跟你說他的導航是廢渣要用他的導航才能顯示左轉右轉和路名以前用很久放還行不過用久了像深灰色款的會慢慢變淡一點我用了快五年了持續使用中我覺得很好用用過一款吋包超過十年很滿意個人頗推這個牌子好看又耐用上次買到折以後約年還在用這個沒有充電呀我剛好有買這款因為有三個插槽才買的質感還算不錯平常會外接螢幕使用會熱但不至於到燙跟充電都不會影響'

In [18]:
count = {}

In [19]:
for char in egg:
    if char in count.keys():
        count[char] += 1
    else:
        count[char] = 1   

In [20]:
count

{'蠻': 1,
 '穩': 2,
 '定': 1,
 '的': 9,
 '行': 2,
 '車': 3,
 '記': 3,
 '錄': 3,
 '器': 1,
 '就': 2,
 '是': 3,
 '要': 5,
 '其': 2,
 '他': 3,
 '次': 2,
 '流': 1,
 '媒': 1,
 '體': 1,
 '不': 8,
 '錯': 4,
 '晚': 2,
 '上': 3,
 '又': 2,
 '清': 1,
 '楚': 1,
 '最': 1,
 '近': 1,
 '有': 7,
 '裝': 4,
 '畫': 1,
 '質': 2,
 '之': 1,
 '前': 3,
 '用': 11,
 '兩': 2,
 '年': 4,
 '也': 1,
 '沒': 2,
 '遇': 1,
 '到': 4,
 '問': 1,
 '題': 1,
 '後': 5,
 '鏡': 3,
 '頭': 1,
 '一': 5,
 '直': 2,
 '斷': 1,
 '線': 2,
 '拉': 1,
 '很': 5,
 '注': 1,
 '意': 2,
 '麻': 1,
 '煩': 1,
 '只': 2,
 '這': 5,
 '種': 1,
 '望': 1,
 '遠': 1,
 '才': 3,
 '拍': 1,
 '邁': 2,
 '現': 2,
 '在': 3,
 '新': 1,
 '會': 5,
 '出': 1,
 '憶': 2,
 '卡': 2,
 '損': 1,
 '壞': 1,
 '語': 1,
 '音': 1,
 '停': 1,
 '止': 1,
 '了': 4,
 '重': 1,
 '知': 1,
 '道': 1,
 '台': 2,
 '視': 3,
 '含': 1,
 '夜': 1,
 '光': 1,
 '六': 1,
 '月': 1,
 '家': 1,
 '裡': 1,
 '老': 1,
 '都': 3,
 '外': 2,
 '目': 1,
 '正': 1,
 '常': 2,
 '便': 1,
 '宜': 1,
 '好': 4,
 '選': 1,
 '擇': 1,
 '我': 4,
 '跟': 2,
 '你': 1,
 '說': 1,
 '導': 2,
 '航': 2,
 '廢': 1,
 '渣': 1,
 '能': 1,
 '顯': 1,
 '示': 1,


### 05. 給每個字一個編號

現在我們要給每個字一個 1, 2, 3 這樣的編號。Shannon 的 Information Theory 告訴我們, 越常出現的字, 應該排在越前面。於是我們需要對前面 `count` 字典中的 values (也就是字出現的次數) 排序。

In [22]:
sorted(count, key=count.get,  reverse=True)

['用',
 '的',
 '不',
 '有',
 '要',
 '後',
 '一',
 '很',
 '這',
 '會',
 '錯',
 '裝',
 '年',
 '到',
 '了',
 '好',
 '我',
 '個',
 '車',
 '記',
 '錄',
 '是',
 '他',
 '上',
 '前',
 '鏡',
 '才',
 '在',
 '視',
 '都',
 '還',
 '過',
 '款',
 '買',
 '穩',
 '行',
 '就',
 '其',
 '次',
 '晚',
 '又',
 '質',
 '兩',
 '沒',
 '直',
 '線',
 '意',
 '只',
 '邁',
 '現',
 '憶',
 '卡',
 '台',
 '外',
 '常',
 '跟',
 '導',
 '航',
 '轉',
 '以',
 '久',
 '慢',
 '使',
 '充',
 '電',
 '蠻',
 '定',
 '器',
 '流',
 '媒',
 '體',
 '清',
 '楚',
 '最',
 '近',
 '畫',
 '之',
 '也',
 '遇',
 '問',
 '題',
 '頭',
 '斷',
 '拉',
 '注',
 '麻',
 '煩',
 '種',
 '望',
 '遠',
 '拍',
 '新',
 '出',
 '損',
 '壞',
 '語',
 '音',
 '停',
 '止',
 '重',
 '知',
 '道',
 '含',
 '夜',
 '光',
 '六',
 '月',
 '家',
 '裡',
 '老',
 '目',
 '正',
 '便',
 '宜',
 '選',
 '擇',
 '你',
 '說',
 '廢',
 '渣',
 '能',
 '顯',
 '示',
 '左',
 '右',
 '和',
 '路',
 '名',
 '放',
 '像',
 '深',
 '灰',
 '色',
 '變',
 '淡',
 '點',
 '快',
 '五',
 '持',
 '續',
 '中',
 '覺',
 '得',
 '吋',
 '包',
 '超',
 '十',
 '滿',
 '人',
 '頗',
 '推',
 '牌',
 '子',
 '看',
 '耐',
 '折',
 '約',
 '呀',
 '剛',
 '因',
 '為',
 '三',
 '插',
 '槽',
 '感',
 '算',
 '平'

於是我們只要「照順序」給 1, 2, 3, ... 就好了。Python 可不可以直接做這件事呢? 用 `enumerate` 是一個方式。 

In [23]:
egg = sorted(count, key=count.get,  reverse=True)

In [24]:
for i, char in enumerate(egg, 1):
    print(char, i)

用 1
的 2
不 3
有 4
要 5
後 6
一 7
很 8
這 9
會 10
錯 11
裝 12
年 13
到 14
了 15
好 16
我 17
個 18
車 19
記 20
錄 21
是 22
他 23
上 24
前 25
鏡 26
才 27
在 28
視 29
都 30
還 31
過 32
款 33
買 34
穩 35
行 36
就 37
其 38
次 39
晚 40
又 41
質 42
兩 43
沒 44
直 45
線 46
意 47
只 48
邁 49
現 50
憶 51
卡 52
台 53
外 54
常 55
跟 56
導 57
航 58
轉 59
以 60
久 61
慢 62
使 63
充 64
電 65
蠻 66
定 67
器 68
流 69
媒 70
體 71
清 72
楚 73
最 74
近 75
畫 76
之 77
也 78
遇 79
問 80
題 81
頭 82
斷 83
拉 84
注 85
麻 86
煩 87
種 88
望 89
遠 90
拍 91
新 92
出 93
損 94
壞 95
語 96
音 97
停 98
止 99
重 100
知 101
道 102
含 103
夜 104
光 105
六 106
月 107
家 108
裡 109
老 110
目 111
正 112
便 113
宜 114
選 115
擇 116
你 117
說 118
廢 119
渣 120
能 121
顯 122
示 123
左 124
右 125
和 126
路 127
名 128
放 129
像 130
深 131
灰 132
色 133
變 134
淡 135
點 136
快 137
五 138
持 139
續 140
中 141
覺 142
得 143
吋 144
包 145
超 146
十 147
滿 148
人 149
頗 150
推 151
牌 152
子 153
看 154
耐 155
折 156
約 157
呀 158
剛 159
因 160
為 161
三 162
插 163
槽 164
感 165
算 166
平 167
接 168
螢 169
幕 170
熱 171
但 172
至 173
於 174
燙 175
影 176
響 177


我們把這做成一個字典。注意字點可以用很類似 list comprehension 的方式去定義。

In [25]:
sorted_char = {char: i for i, char in enumerate(egg, 1)}

試用一下!

In [26]:
sorted_char['的']

2

記得我們前面說要把空白對到 0。

In [27]:
sorted_char[" "] = 0

於是我們字的對應就做好了!

### 06. 把一句話用編碼顯示

終於到了最後的時刻, 我們來把每句話編碼一下。記得前面我們範例有一個句子。

In [31]:
X

'後鏡頭一直斷線 拉線要很注意 很麻煩'

要把這一一變成對應的數字, 要怎麼做呢? 一個方式是我們定義一個函數, 輸入是一個字, 輸出就是對應的編碼。也就是說:

    char --> sorted_char[char]
    
要把這個函數用在我們的句字上, 就用 `map` 指令。

In [32]:
list(map(lambda char:sorted_char[char], X))

[6, 26, 82, 7, 45, 83, 46, 0, 84, 46, 5, 8, 85, 47, 0, 8, 86, 87]

WOW! 這太炫了。我們終於可以完成我們的輸入部份。

In [33]:
x = []

for review in x_tmp:
    record = list(map(lambda char:sorted_char[char], review))
    x.append(record)

再來做正確答案 y, 這就簡單多了。

In [37]:
y = df["正負評"].values

In [38]:
y

array([1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1])